# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [17]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, desc, min, max, split, udf
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, StandardScaler, StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re

In [18]:
# create a Spark session
spark = SparkSession.builder\
                    .appName('Sparkify')\
                    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [20]:
df = spark.read.json("mini_sparkify_event_data.json")

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [25]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [26]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [27]:
df.select('userId').show()

+------+
|userId|
+------+
|    30|
|     9|
|    30|
|     9|
|    30|
|     9|
|     9|
|    30|
|    30|
|    30|
|     9|
|     9|
|    30|
|     9|
|     9|
|    30|
|     9|
|    74|
|    30|
|     9|
+------+
only showing top 20 rows



In [28]:
df = df.where("userId != '' and userId != ' '")

In [29]:
df = df.dropna(how='any', subset=['userId', 'sessionId'])

In [30]:
valid = df.dropDuplicates()

In [31]:
valid.groupby('userId').max().select('userId').sort('userId').show()

+------+
|userId|
+------+
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
|100018|
+------+
only showing top 20 rows



In [32]:
valid.select('page').dropDuplicates().sort('page').show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|              Logout|
|            NextSong|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+



In [33]:
churn = udf(lambda x: 1 if x == 'Cancellation Confirmation' else 0, IntegerType())

In [34]:
valid = valid.withColumn('churn', churn(valid.page))

In [35]:
valid.where("churn == 1").count()

52

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [36]:
valid.groupby(["churn", 'gender']).count().show()

+-----+------+------+
|churn|gender| count|
+-----+------+------+
|    1|     F|    20|
|    0|     M|123544|
|    1|     M|    32|
|    0|     F|154558|
+-----+------+------+



In [37]:
gender_idx = StringIndexer(inputCol="gender", outputCol='gender_idx')
page_idx = StringIndexer(inputCol="page", outputCol='page_idx')
gender_encoder = OneHotEncoder(inputCol='gender_idx', outputCol='gender_vec')
page_encoder = OneHotEncoder(inputCol='page_idx', outputCol='page_vec')
assembler = VectorAssembler(inputCols=["gender_vec", "page_vec", "status"], outputCol="features")
idx = StringIndexer(inputCol="churn", outputCol="label")

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [38]:
model =  LogisticRegression(maxIter=15)
pipeline = Pipeline(stages=[gender_idx, page_idx, gender_encoder, page_encoder, assembler, idx, model])

In [39]:
rest, validation = valid.randomSplit([0.9, 0.1], seed=42)

In [40]:
param = ParamGridBuilder() \
    .addGrid(model.regParam,[0.0, 0.1, ]) \
    .addGrid(model.elasticNetParam,[0.0, 0.1, 0.5, 1.0]) \
    .build()


cross = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=param,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)

In [41]:
fitted = cross.fit(rest)

In [42]:
results = fitted.transform(validation)

In [43]:
results.select("rawPrediction","probability", "prediction").show()

+--------------------+--------------------+----------+
|       rawPrediction|         probability|prediction|
+--------------------+--------------------+----------+
|[21.6661123683481...|[0.99999999961048...|       0.0|
|[21.6640372010717...|[0.99999999960967...|       0.0|
|[21.6657777009774...|[0.99999999961035...|       0.0|
|[21.6640372010717...|[0.99999999960967...|       0.0|
|[17.7169719283065...|[0.99999997978764...|       0.0|
|[17.3379741456721...|[0.99999997047339...|       0.0|
|[21.2936050554995...|[0.99999999943466...|       0.0|
|[21.2936050554995...|[0.99999999943466...|       0.0|
|[21.6640372010717...|[0.99999999960967...|       0.0|
|[17.7107117119453...|[0.99999997966071...|       0.0|
|[17.3465397827343...|[0.99999997072523...|       0.0|
|[17.3465397827343...|[0.99999997072523...|       0.0|
|[18.0933973655363...|[0.99999998612805...|       0.0|
|[18.4638295111085...|[0.99999999042231...|       0.0|
|[18.0933973655363...|[0.99999998612805...|       0.0|
|[18.46382

In [44]:
fitted.avgMetrics

[1.0,
 1.0,
 1.0,
 1.0,
 0.9996942325087543,
 0.9996942325087543,
 0.9996942325087543,
 0.9996942325087543]

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.